In [14]:
import pandas as pd
import pylab as pl
from IPython import embed
from ipywidgets import HTML
from ipywidgets.embed import embed_minimal_html
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles, Circle, Polyline, Heatmap
import numpy as np
import json

trips = pd.read_csv(
    "../data/bysykkel/trips-2016.10.1-2016.10.31.csv",
    sep=",",
    parse_dates=["Start time", "End time"],
    date_parser=lambda col: pd.to_datetime(col, utc=True),
)
trips = trips.dropna(subset=["End station"]).astype({"End station": int})
trips["duration"] = trips["End time"] - trips["Start time"]

with open("../data/bysykkel/stations.json") as f:
    stations_json = json.load(f)

station_json = stations_json['stations']
stations = pd.json_normalize(station_json)
stations = stations.set_index('id')
stations = stations.drop(["bounds", "subtitle"], axis=1)   # axis=0 drops rows
merged_bike_stats = pd.merge(
    trips, stations, how="left", left_on="Start station", right_index=True
)
merged_bike_stats


,Start station,Start time,End station,End time,duration,in_service,title,number_of_locks,center.latitude,center.longitude
0,283,2016-10-01 04:00:08+00:00,238,2016-10-01 04:09:47+00:00,0 days 00:09:39,True,Marienlyst,12.0,59.932449,10.721813
1,169,2016-10-01 04:00:41+00:00,175,2016-10-01 04:11:07+00:00,0 days 00:10:26,True,Bogstadveien,18.0,59.924729,10.724579
2,211,2016-10-01 04:01:02+00:00,162,2016-10-01 04:15:52+00:00,0 days 00:14:50,True,Oslo City,12.0,59.912779,10.751116
3,200,2016-10-01 04:01:20+00:00,163,2016-10-01 04:15:04+00:00,0 days 00:13:44,True,Tøyen skole,15.0,59.914990,10.773942
4,157,2016-10-01 04:02:16+00:00,163,2016-10-01 04:04:29+00:00,0 days 00:02:13,True,Nylandsveien,30.0,59.915620,10.762248
...,...,...,...,...,...,...,...,...,...,...
242753,284,2016-10-31 22:53:30+00:00,164,2016-10-31 23:04:13+00:00,0 days 00:10:43,True,Majorstuen skole,27.0,59.929829,10.713032
242754,218,2016-10-31 22:53:48+00:00,292,2016-10-31 23:09:17+00:00,0 days 00:15:29,True,Briskeby,15.0,59.920187,10.717955
242755,204,2016-10-31 22:54:28+00:00,277,2016-10-31 23:01:59+00:00,0 days 00:07:31,True,Paléhaven,48.0,59.910287,10.750012
242756,204,2016-10-31 22:56:57+00:00,201,2016-10-31 23:02:16+00:00,0 days 00:05:19,True,Paléhaven,48.0,59.910287,10.750012


In [18]:

oslo_center = 59.9127, 10.7461    #NB ipyleaflet uses Lat-Long (i.e. y,x, when specifying coordinates)
oslo_map = Map(basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),center=oslo_center, zoom=13)


def add_markers(row):
    # in this function the variable row represents one row (station_data.iloc) in a loop
    center = row['center.latitude'], row['center.longitude'] 
    marker = Marker(location=center, title=row['title'])

    #m.popup = HTML(f"{marker.title}<br/>ID: {row['start_station_id']}")

    oslo_map.add_layer(marker)
    


def add_lines(row):
    start = row['center.latitude'], row['center.longitude']
    #end = # decide on an endpoint of a line
    l = Polyline(locations=[start,end], color="green", fill=False)
    oslo_map.add_layer(l)
    circle.popup = HTML(f"{row['title']} <")


def add_one_line():
    station0 = stations[stations["title"] == "Storo Storsenter"]
    station1 = stations[stations["title"] == "BI Nydalen"]
    start = float(station0["center.latitude"]), float(station0["center.longitude"])
    end = float(station1["center.latitude"]), float(station1["center.longitude"])
    l = Polyline(locations=[start,end], color="green", fill=False)
    oslo_map.add_layer(l)
    

def add_heatmap_two_stations():
    name = "BI Nydalen"
    location = stations[stations["title"] == name]
    l0,l1 = float(location["center.latitude"]), float(location["center.longitude"])
    intensity = (merged_bike_stats[merged_bike_stats["title"] == name].count())[0]
    
    max_intensity = 1592
    scale = 40     # seems to be the "maximal" intensity on the heat image
                   # scale may need to be changed manually
    
    gradient = {0.4: "blue", 0.6: "cyan", 0.7: "lime", 0.8: "yellow", 1.0: "red"}
    
    locations = [l0,l1,intensity]
    
    name2 = "Storo Storsenter"
    location2 = stations[stations["title"] == name2]
    l02,l12 = float(location2["center.latitude"]), float(location2["center.longitude"])
    intensity2 = (merged_bike_stats[merged_bike_stats["title"] == name2].count())[0]
    locations2 = [l02,l12,intensity2]
    
    heat = Heatmap(locations=[[l0,l1,intensity/max_intensity*scale],[l02, l12, intensity2/max_intensity*scale]], radius=20, blur=10,gradient=gradient)
    oslo_map.add_layer(heat)

    
#stations.apply(add_markers, axis=1) #Adds markers to all bike stations
add_one_line()
add_heatmap_two_stations()
oslo_map


Map(center=[59.9127, 10.7461], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…